In [ ]:
# Install PyGEL; note does finish the installation on linux.
# If on linux, go download the source and build it to get the missing libPyGEL3D.so
!pip install PyGEL3D

In [ ]:
# Import PyGEL modules
from PyGEL3D import gel
from PyGEL3D import js
import numpy as np
js.set_export_mode()

## Experiment with a single obj file

In [ ]:
# Load a mesh and compute its distance field
m = gel.obj_load("../data/cochleas/shape18_pca.obj")

In [ ]:
# View the object
viewer = gel.GLManifoldViewer()
viewer.display(m, mode='n', bg_col=[1, 1, 1])
del viewer

In [ ]:
# Compute distance field
mdist = gel.MeshDistance(m)

In [ ]:
mdist.signed_distance([-0, 0, 0])

In [ ]:
# Discretize the distance field
resolution = 64
boundary = -10
width = 20
step_size = width/resolution

output = []

for i in range(1,resolution+1):
    for j in range(1,resolution+1):
        for k in range(1,resolution+1):
            is_inside = mdist.ray_inside_test([boundary+step_size*i, boundary+step_size*j, boundary+step_size*k])
            if is_inside:
                output.append([i,j,k])

                
output = np.array(output)
np.savetxt(file+".txt", output, delimiter=",")

## Compute discrete distance field text files for range of cochleas.

In [2]:
# Prepare the file paths and names.

cochlea_folder = "../data/cochleas/"
cochlea_files = [
#     "shape05_pca",
#     "shape06_pca",
#     "shape08_pca",
#     "shape09_pca",
#     "shape10_pca",
#     "shape11_pca",
#     "shape12_pca",
#     "shape15_pca",
#     "shape16_pca",
    "shape18_pca",
    "shape19_pca",
#     "shape20_pca",
#     "shape21_pca",
    "shape22_pca",
#     "shape23_pca",
#     "shape24_pca",
#     "shape5876_pca",
#     "shape6317_pca"
]

cochlea_files


['shape18_pca', 'shape19_pca', 'shape22_pca']

In [ ]:
# Load meshes.
cochlea_indices = range(len(cochlea_files))
cochlea_meshes = []
for cochlea_index in cochlea_indices:
    print("("+str(cochlea_index+1)+"/"+str(len(cochlea_indices))+") "+"Loading mesh from \""+cochlea_files[cochlea_index]+".obj\"...")
    cochlea_meshes.append(gel.obj_load(cochlea_folder+cochlea_files[cochlea_index]+".obj"))
print("--------------------------------------\nFinished loading meshes from "+str(len(cochlea_indices))+" files.")

In [ ]:
# Compute distance fields.
cochlea_distance_fields = []
for cochlea_index in range(len(cochlea_meshes)):
    print("("+str(cochlea_index+1)+"/"+str(len(cochlea_meshes))+") "+"Generating distance field for \""+cochlea_files[cochlea_index]+".obj\"...")
    cochlea_distance_fields.append(gel.MeshDistance(cochlea_meshes[cochlea_index]))
print("--------------------------------------\nFinished computing distance fields for "+str(len(cochlea_meshes))+" meshes.")

In [ ]:
# Voxelize distance fields.
resolution = 64
boundary = [-10, -10, -10]
width = 20
step_size = width/resolution
for cochlea_index in range(len(cochlea_distance_fields)):
    output = []
    print("("+str(cochlea_index+1)+"/"+str(len(cochlea_distance_fields))+") "+"Voxelizing "+cochlea_files[cochlea_index]+" to "+str(resolution)+"x"+str(resolution)+"x"+str(resolution)+" grid...")
    for i in range(1,resolution+1):
        if (i % 10 == 0):
            print("  "+str(round(i/resolution*100))+"% done...")
        for j in range(1,resolution+1):
            for k in range(1,resolution+1):
                is_inside = cochlea_distance_fields[cochlea_index].ray_inside_test([-boundary[0]-step_size*i, -boundary[1]-step_size*j, -boundary[2]-step_size*k])
                if is_inside:
                    output.append([i,j,k])
    output = np.array(output)
    output_file_name = cochlea_files[cochlea_index]+"_"+str(resolution)+".txt";
    np.savetxt(cochlea_folder+output_file_name, output, delimiter=",")
    print("  100% done. Output saved to \""+output_file_name+"\".")
print("--------------------------------------\nFinished voxelizing "+str(len(cochlea_distance_fields))+" distance fields.")

## Big all-in-one computation. Very heavy...

In [ ]:
# Compute discrete distance field txt files for range of cochleas

mirror = True
resolution = 64
boundary = -10
width = 20
step_size = width/resolution
cochlea_indices = range(len(cochlea_files))

for file_index in cochlea_indices:
    print("("+str(file_index+1)+"/"+str(len(cochlea_indices))+") "+"Loading mesh from \""+cochlea_files[file_index]+".obj\"...")
    m = gel.obj_load(cochlea_folder+cochlea_files[file_index]+".obj")
    print("Creating distance field...")
    mdist = gel.MeshDistance(m)
    output = []
    print("Iterating over "+str(resolution)+"x"+str(resolution)+"x"+str(resolution)+" grid:")
    for i in range(1,resolution+1):
        if (i % 10 == 0):
            print("  "+str(i/resolution*100)+"% done...")
        for j in range(1,resolution+1):
            for k in range(1,resolution+1):
                if mirror
                    is_inside = mdist.ray_inside_test([boundary+step_size*i, boundary+step_size*j, boundary+step_size*k])
                else
                    is_inside = mdist.ray_inside_test([-boundary-step_size*i, -boundary-step_size*j, -boundary-step_size*k])
                if is_inside:
                    output.append([i,j,k])

    output = np.array(output)
    output_file_name = cochlea_folder+cochlea_files[file_index]+"_"+str(resolution)
    if mirror
        output_file_name += "_mirrored"
    output_file_name += ".txt"
    np.savetxt(output_file_name, output, delimiter=",")
    print("  100% done.")
    print("Voxelized mesh saved to "+output_file_name+".")
    print("-------------------------------------------------------------------")
    del m, mdist
print("Finishes voxelizing "+str(len(cochlea_indices))+" meshes.")


## Special preprocessing based on the distance field

Below the code generates a voxel field, but instead of setting each entry to 1 it will be set to

$$
P(x) = \exp(-d_\alpha(x)) + \beta,
$$

where $\alpha$ is the clamp value and $\beta$ is the smallest value before normalisation.

In [ ]:
def clamp(x,alpha):
    alpha = abs(alpha)
    if x > alpha:
        return alpha
    elif x < -alpha:
        return -alpha
    return x

In [ ]:
resolution = 64
boundary = -12
width = 20
step_size = width/resolution
cochlea_indices = range(len(cochlea_files))
# cochlea_indices = range(0,1)

sign = 1;

for file_index in cochlea_indices:
    print("("+str(file_index+1)+"/"+str(len(cochlea_indices))+") ----------")
    print("  Loading mesh from \""+cochlea_files[file_index]+".obj\"...")
    m = gel.obj_load(cochlea_folder+cochlea_files[file_index]+".obj")
    print("  Creating distance field...")
    mdist = gel.MeshDistance(m)
    output = []
    print("  Iterating over "+str(resolution)+"x"+str(resolution)+"x"+str(resolution)+" grid:")
    for i in range(1,resolution+1):
        if (i % 10 == 0):
            print("    "+str(round(i/resolution*100))+"% done...")
        for j in range(1,resolution+1):
            for k in range(1,resolution+1):
                p = [-boundary-step_size*i, -boundary-step_size*j, -boundary-step_size*k]
                d = mdist.signed_distance(p)
                if (i == 1 and j == 1 and k == 1):
                    if d < 0:
                        sign = -1;
                    print("    (sign of distance = "+str(sign)+")")
                output.append([i,j,k,sign*d])
    
    output = np.array(output)
    output_file_name = cochlea_folder+cochlea_files[file_index]+"_"+str(resolution)+"_signed_distance"
    output_file_name += ".txt"
    np.savetxt(output_file_name, output, delimiter=",")
    print("    100% done.")
    print("  Voxelized mesh saved to "+output_file_name+".")
    del m, mdist, output
    sign = 1
print("Finished voxelizing "+str(len(cochlea_indices))+" meshes.")